In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy import text


In [2]:
# # workshoplist_df = pd.read_csv('workshop_list.csv')
# # url_list = workshoplist_df["Workshop Seite Eversports"]
# # url_list
# url = ['''

# https://www.eversports.de/e/workshop/x31ysLeOS
# https://www.eversports.de/e/workshop/x1w0Mq3Y7
# https://www.eversports.de/e/workshop/xFwPArnAH
# https://www.eversports.de/e/workshop/Sh6fqaIks
# https://www.eversports.de/e/workshop/feME-_7tG
# https://www.eversports.de/e/workshop/dCR2PVd05
# https://www.eversports.de/e/workshop/nb8_WV2wH
# https://www.eversports.de/e/workshop/cDPxswQQp
# https://www.eversports.de/e/workshop/xo_EQbOQn
# https://www.eversports.de/e/workshop/HGkpQ9uAJ
# https://www.eversports.de/e/workshop/Bz5neTPTa
# https://www.eversports.de/e/workshop/3zdELX6G_
# https://www.eversports.de/e/workshop/tUZ-pbNF0
# https://www.eversports.de/e/workshop/5u5RfrpDC
# https://www.eversports.de/e/workshop/RPiNq9mm6

# ''']
# didi_list = [
# 0     https://www.eversports.de/e/workshop/v4j4ZnWFW
# 1     https://www.eversports.de/e/workshop/cGbDoasJb
# 2     https://www.eversports.de/e/workshop/xv2_PAISE
# 3     https://www.eversports.de/e/workshop/ocz15aUrk x
# 4     https://www.eversports.de/e/workshop/tx-_485PK
# 5     https://www.eversports.de/e/workshop/nDyryrDzB
# 6     https://www.eversports.de/e/workshop/WIiSb5dg1
# 7     https://www.eversports.de/e/workshop/OjGR_JBOx
# 8     https://www.eversports.de/e/workshop/8FXiuS8_E
# 9     https://www.eversports.de/e/workshop/bzRPpiHhj
# 10    https://www.eversports.de/e/workshop/0CWbuWeCP
# 11    https://www.eversports.de/e/workshop/PG7A7dCx6
# 12    https://www.eversports.de/e/workshop/R3VflUB-G
# 13    https://www.eversports.de/e/workshop/6HjnBPeOE
# 14    https://www.eversports.de/e/workshop/vShOvMLtb
# 15    https://www.eversports.de/e/workshop/7rFKxizij
# 16    https://www.eversports.de/e/workshop/XVTRcbEDY
# 17    https://www.eversports.de/e/workshop/uLOMWc_ox
# 18    https://www.eversports.de/e/workshop/qWwvQ8wm6
# ]

In [3]:
url = "https://www.eversports.de/e/workshop/x31ysLeOS"

In [4]:
# def workshop_overview_func(url):

# Send a GET request to the webpage
response = requests.get(url)
# Parse the HTML content of the webpage
soup = BeautifulSoup(response.content, 'html.parser')
response

# extract workshop name
workshop_name = soup.find(
     'div', class_='header-holder').find('h3').text.strip()

workshop_beschreibung = ''.join([p.text.strip() for p in soup.find(
      'div', class_='marketplace-truncated-text-block js_truncate-block').find_all('p')])

# extract pole studio name
pole_studio = soup.find(
       'a', class_='venue-name action-link-default').text.strip()

# extract pole studio address
try:
    stadt = soup.find('div', class_='venue-location').text.strip()
except AttributeError:
        stadt = 0

# extract pole studio street address
try:
    strasse = soup.find('div', class_='venue-location').text.strip()
except AttributeError:
    strasse = 0

# extract date
datum = soup.find('div', class_='date-holder').text.strip()

# extract time
uhrzeit = soup.find('div', class_='date-holder').text.strip()

# extract duration
dauer = soup.find('div', class_='date-holder').text.strip()

# extract level
try:
    level = soup.find('div', class_='event-level').text.strip()
except AttributeError:
    level = 0

# extract trainer
trainer = soup.find('div', class_='trainer-name').text.strip()

# extract trainer description
try:    
    trainer_beschreibung = soup.find('div', class_='about-teacher text').text.strip()
except:
    trainer_beschreibung = 0

# extract price
preis = soup.find('div', class_='event-detail-price__price').text.strip()

# Reconstruct Eversports Pole Studio Webpage
a_tag = soup.find('a', {'class': 'venue-name'})

link_p2 = a_tag['href']
link_p1 = "https://www.eversports.de"
polestudio_eversports_seite = link_p1 + link_p2


# create pandas DataFrame
workshop_overview_df = pd.DataFrame({
        'Workshop Name': [workshop_name],
        'Workshop Beschreibung': [workshop_beschreibung],
        'PoleStudio': [pole_studio],
        'Stadt': [stadt],
        'Straße': [strasse],
        'Datum': [datum],
        'Uhrzeit': [uhrzeit],
        'Dauer': [dauer],
        'Level': [level],
        'Trainer': [trainer],
        'Trainer Beschreibung': [trainer_beschreibung],
        'Preis': [preis],
        'Workshop Everest Seite': url,
        'PoleStudio Eversports Seite': polestudio_eversports_seite
    })

workshop_overview_df['Version'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Splitt the string to get the town
try:
    workshop_overview_df["Stadt"] = stadt.split(",")[1].split(" ")[2]
except:
    workshop_overview_df["Stadt"] = 0

# Splitt the string to get the street
try:
    workshop_overview_df["Straße"] = strasse[5:-7]
except:
    workshop_overview_df["Straße"] = 0

#
workshop_overview_df["Datum"] = datum[3:13]
#
workshop_overview_df["Uhrzeit"] = uhrzeit[14:29]

#
try:
    workshop_overview_df["Dauer"] = dauer[-7:-1]
except:
    workshop_overview_df["Dauer"] = 0
#
try:
    workshop_overview_df["Level"] = level[5:]
except:
    workshop_overview_df["Level"] = 0

workshop_overview_df

,Workshop Name,Workshop Beschreibung,PoleStudio,Stadt,Straße,Datum,Uhrzeit,Dauer,Level,Trainer,Trainer Beschreibung,Preis,Workshop Everest Seite,PoleStudio Eversports Seite,Version
0,Hip Hop Pole Workshop,In diesem Workshop werden Hip Hop Dance Moves ...,poda Studio,Aachen,"10-12 Martinstraße Innenhof, 52062",30.04.2023,20:15-21:45 Uhr,90 min,Alle Level,Lara,"Offen, Engagiert & Floorworkfanatiker\nLara ta...","Preis: 35,00 €",https://www.eversports.de/e/workshop/x31ysLeOS,https://www.eversports.de/s/poda-studio,2023-04-18 16:20:01


In [5]:
# create the "CSV" directory if it does not exist
if not os.path.exists("Workshop_CSV"):
     os.makedirs("Workshop_CSV")

# save the CSV file in the "CSV" directory
workshop_overview_df.to_csv(f"Workshop_CSV/{workshop_name}.csv", index=False)

Database SQL

In [6]:
#SQL
# host = 'Your information!'
# schema = 'hallofpole'
# user = 'Your information!'
# password = 'Your information!'
# port = Your information!

# # create connection string with the database name included
# con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# # create sqlalchemy engine
# engine = create_engine(con)


In [7]:
# # Write the DataFrame to the SQL database
# workshop_overview_df.to_sql(name='workshop_overview', con=con, if_exists='replace', index=False)


In [8]:
# #Read SQL and create a CSV

# # create sqlalchemy engine
# engine = create_engine(con)

# # read table from SQL database into a DataFrame
# workshop_overview_df = pd.read_sql_table('workshop_overview', con=engine)

# # write DataFrame to CSV file
# workshop_overview_df.to_csv('workshop_overview.csv', index=False)
